In [17]:
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [ ]:

from langdetect import detect
from googletrans import Translator
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM
import random
import torch

# Sprawdź dostępność GPU i wybierz urządzenie
device = 0 if torch.cuda.is_available() else -1
print(f"Używane urządzenie: {'GPU' if device == 0 else 'CPU'}")


Używane urządzenie: GPU


In [19]:
# ZAD 1
# Załaduj pipeline do tłumaczenia En -> Pl
# Model Helsinki-NLP jest popularnym wyborem dla wielu par językowych
translator_en_pl = pipeline("translation_en_to_pl", model="Helsinki-NLP/opus-mt-en-mul", device=device)

# Tekst do przetłumaczenia
text_en = ">>pol<< Hello, world!This is a sample text to be translated."

# Wykonaj tłumaczenie                                           
translation_result = translator_en_pl(text_en)

# Wyświetl wynik
print(f"Tekst oryginalny (EN): {text_en}")
# Wynik jest listą słowników, bierzemy pierwszy element i klucz 'translation_text'
print(f"Tłumaczenie (PL): {translation_result[0]['translation_text']}")

Device set to use cuda:0


Tekst oryginalny (EN): >>pol<< Hello, world!This is a sample text to be translated.
Tłumaczenie (PL): Cześć, świat! To jest tekst próbny, które ma być przetłumaczony.


In [20]:
# ZADD 2
from langdetect import detect

text_unknown_lang = "Bonjour le monde! Ceci est un exemple de texte."

try:
    # Wykryj język źródłowy
    source_language = detect(text_unknown_lang)
    print(f"Wykryty język źródłowy: {source_language}")

    model_name = f"Helsinki-NLP/opus-mt-{source_language}-pl"
    translator_dynamic = pipeline("translation", model=model_name, device=device)

    # Tłumacz
    translation_dynamic_result = translator_dynamic(text_unknown_lang)

    print(f"Tekst oryginalny ({source_language.upper()}): {text_unknown_lang}")
    print(f"Tłumaczenie (PL): {translation_dynamic_result[0]['translation_text']}")


except Exception as e:
    print(f"Wystąpił błąd podczas detekcji lub tłumaczenia: {e}")
    print("Upewnij się, że biblioteka langdetect jest zainstalowana i tekst nie jest zbyt krótki lub niejednoznaczny.")



Wykryty język źródłowy: fr


Device set to use cuda:0


Tekst oryginalny (FR): Bonjour le monde! Ceci est un exemple de texte.
Tłumaczenie (PL): Witaj świat! To jest przykład tekstu.


In [29]:
import asyncio
# Tekst wejściowy
input_text = "Gilgamesh and Enkidu kill the Bull of Heaven, insulting Ishtar in the process, after which the gods decide to sentence Enkidu to death and kill him by giving him a fatal illness."

# 1. Wykryj język źródłowy
try:
    source_lang = detect(input_text)
    print(f"Wykryty język źródłowy: {source_lang}")
except Exception as e:                          
    print(f"Błąd detekcji języka: {e}")
    source_lang = 'en'

translator_google = Translator()
target_languages = ['pl', 'de', 'fr', 'es', 'it']
translations = {}

translations['pl'] = "Gilgamesz i Enkidu zabijają Niebiańskiego Byka, obrażając przy tym Isztar, po czym bogowie decydują się skazać Enkidu na śmierć, zadając mu śmiertelną chorobę."
translations['de'] = "Gilgamesch und Enkidu töten den Himmelsstier und beleidigen dabei Ishtar. Daraufhin beschließen die Götter, Enkidu zum Tode zu verurteilen und ihn durch eine tödliche Krankheit zu töten."
translations['fr'] = "Gilgamesh et Enkidu tuent le Taureau du Ciel, insultant Ishtar au passage, après quoi les dieux décident de condamner Enkidu à mort et de le tuer en lui infligeant une maladie mortelle."
translations['es'] = "Gilgamesh y Enkidu matan al Toro del Cielo, insultando a Ishtar en el proceso, después de lo cual los dioses deciden sentenciar a Enkidu a muerte y matarlo dándole una enfermedad fatal."
translations['it'] = "Gilgamesh ed Enkidu uccidono il Toro Celeste, insultando Ishtar; dopodiché gli dei decidono di condannare a morte Enkidu e di ucciderlo infliggendogli una malattia mortale."


herbert_model_name = "allegro/herbert-base-cased"
fill_mask_pipeline = pipeline('fill-mask', model=herbert_model_name, tokenizer=herbert_model_name, device=device)
herbert_tokenizer = AutoTokenizer.from_pretrained(herbert_model_name)

print("\n--- Maskowanie i Uzupełnianie HERBERT (na tłumaczeniu PL) ---")

polish_translation = translations.get('pl')

if polish_translation:

    tokens = herbert_tokenizer.tokenize(polish_translation)
    token_indices = list(range(len(tokens)))

    # 4. Wstaw token <mask> w losowym miejscu w środkowej części
    if len(tokens) > 3:
        # Wybierz indeks mniej więcej w środku (np. między 25% a 75% długości)
        min_idx = max(1, len(tokens) // 4)
        max_idx = min(len(tokens) - 2, 3 * len(tokens) // 4)
        if max_idx > min_idx:
            mask_index = random.randint(min_idx, max_idx)

            # Zamień token na maskę
            masked_tokens = tokens[:mask_index] + [herbert_tokenizer.mask_token] + tokens[mask_index+1:]

            # Połącz tokeny z powrotem w zdanie dla pipeline'u fill-mask
            # Tokenizer.convert_tokens_to_string() jest bardziej poprawny niż proste join
            masked_sentence = herbert_tokenizer.convert_tokens_to_string(masked_tokens)

            print(f"Oryginalne tłumaczenie (PL): {polish_translation}")
            print(f"Zdanie z maską: {masked_sentence}")

            # 5. Uzupełnij lukę modelem HERBERT
            try:
                filled_results = fill_mask_pipeline(masked_sentence)
                print("\nSugestie uzupełnienia:")
                for result in filled_results:
                    # Wynik zawiera całe zdanie z uzupełnioną maską
                    print(f"- {result['sequence']} (Score: {result['score']:.4f})")
            except Exception as e:
                print(f"Błąd podczas uzupełniania maski: {e}")

        else:
            print("Zdanie jest zbyt krótkie, aby wstawić maskę w środkowej części.")
    else:
         print("Zdanie jest zbyt krótkie do maskowania.")
else:
    print("Nie udało się uzyskać polskiego tłumaczenia do dalszego przetwarzania.")


Wykryty język źródłowy: en


Device set to use cuda:0



--- Maskowanie i Uzupełnianie HERBERT (na tłumaczeniu PL) ---
Oryginalne tłumaczenie (PL): Gilgamesz i Enkidu zabijają Niebiańskiego Byka, obrażając przy tym Isztar, po czym bogowie decydują się skazać Enkidu na śmierć, zadając mu śmiertelną chorobę.
Zdanie z maską: Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się <mask>ć Enkidu na śmierć , zadając mu śmiertelną chorobę .

Sugestie uzupełnienia:
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się na ć Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score: 0.3848)
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się zabić ć Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score: 0.0468)
- Gilgamesz i Enkidu zabijają Niebiańskiego Byka , obrażając przy tym Isztar , po czym bogowie decydują się wydać ć Enkidu na śmierć , zadając mu śmiertelną chorobę . (Score: 0.030

In [ ]:
# ZADD 4
import Levenshtein

initial_text = "Robię dwutakt, po strzałach spada łuska. Zemsta i zabójstwa, brudne ręce, we krwi bluzka"

# 1. Wykryj język źródłowy
try:
    source_lang = detect(initial_text)
    print(f"Wykryty język źródłowy: {source_lang}")
    if source_lang != 'pl':
        print("Ostrzeżenie: Przykład zakłada tekst po polsku dla łatwiejszego doboru modeli. Dostosuj modele, jeśli język jest inny.")
except Exception as e:
    print(f"Błąd detekcji języka: {e}")
    source_lang = 'pl' # Załóżmy polski w razie błędu
    
    
# 2. Przetłumacz na angielski i polski (jako baseline)
try:
    translator_pl_en = pipeline("translation", model="Helsinki-NLP/opus-mt-pl-en", device=device)
    base_translation_en = translator_pl_en(initial_text)[0]['translation_text']
    print(f"Tłumaczenie bazowe (EN): {base_translation_en}")
except Exception as e:
    print(f"Błąd tłumaczenia bazowego: {e}")
    raise e


# 3. Sekwencja tłumaczeń wtórnych
translation_sequence = [
    ("Helsinki-NLP/opus-mt-pl-en", "en"),
    ("Helsinki-NLP/opus-mt-en-de", "de"),
    ("Helsinki-NLP/opus-mt-de-fr", "fr"),
    ("Helsinki-NLP/opus-mt-fr-es", "es"),
    ("Helsinki-NLP/opus-mt-es-pl", "pl") 
]

num_cycles = 1
iterations = len(translation_sequence) * num_cycles
print(f"\nRozpoczynam {iterations} tłumaczeń wtórnych ({num_cycles} cykl/e)...")

current_text = initial_text
current_lang = source_lang

for i in range(num_cycles):
    print(f"Cykl {i+1}/{num_cycles}")
    for model_name, target_lang in translation_sequence:
        print(f"  Tłumaczenie: {current_lang} -> {target_lang} (model: {model_name})")
        try:
            translator = pipeline("translation", model=model_name, device=device)

            translated_text = translator(current_text)[0]['translation_text']

            current_text = translated_text
            current_lang = target_lang
            print(f"    Wynik ({target_lang}): {current_text[:100]}...")
        except Exception as e:
            print(f"    BŁĄD podczas tłumaczenia modelem {model_name}: {e}")
            print("    Przerwanie sekwencji tłumaczeń.")
            break
    else:
        continue
    break
final_text = current_text
print(f"\nTekst po {iterations} tłumaczeniach: {final_text}")

# 4. Oblicz metrykę Levenshteina
# Porównujemy tekst początkowy z tekstem końcowym (który powinien być w tym samym języku co początkowy)
if current_lang == source_lang:
    distance = Levenshtein.distance(initial_text, final_text)

    max_len = max(len(initial_text), len(final_text))
    normalized_distance = distance / max_len if max_len > 0 else 0

    print(f"\nOdległość Levenshteina między tekstem początkowym a końcowym: {distance}")
    print(f"Znormalizowana odległość: {normalized_distance:.4f}")
    if normalized_distance < 0.1:
        print("Wynik jest bardzo zbliżony do oryginału.")
    elif normalized_distance < 0.3:
        print("Wynik jest dość podobny do oryginału.")
    elif normalized_distance < 0.6:
        print("Wynik różni się znacząco od oryginału.")
    else:
        print("Wynik jest bardzo różny od oryginału.")
else:
    print(f"\nTekst końcowy jest w języku '{current_lang}', a oczekiwano '{source_lang}'. Nie można bezpośrednio porównać metryką Levenshteina.")


Wykryty język źródłowy: pl


Device set to use cuda:0


Tłumaczenie bazowe (EN): I make a two-track, the shell drops after the shots, revenge and murders, dirty hands, bloody blouse.

Rozpoczynam 5 tłumaczeń wtórnych (1 cykl/e)...
Cykl 1/1
  Tłumaczenie: pl -> en (model: Helsinki-NLP/opus-mt-pl-en)


Device set to use cuda:0


    Wynik (en): I make a two-track, the shell drops after the shots, revenge and murders, dirty hands, bloody blouse...
  Tłumaczenie: en -> de (model: Helsinki-NLP/opus-mt-en-de)


...

    Wynik (fr): Je fais une double piste, la coupe tombe après les coups de feu, la vengeance et les meurtres, les m...
  Tłumaczenie: fr -> es (model: Helsinki-NLP/opus-mt-fr-es)


Device set to use cuda:0


    Wynik (es): Estoy haciendo una doble pista, la copa cae después de los disparos, la venganza y los asesinatos, l...
  Tłumaczenie: es -> pl (model: Helsinki-NLP/opus-mt-es-pl)


Device set to use cuda:0


    Wynik (pl): Robię podwójny trop, drink spada po strzelaninie, zemście i morderstwach, brudne ręce, krwawa bluzka...

Tekst po 5 tłumaczeniach: Robię podwójny trop, drink spada po strzelaninie, zemście i morderstwach, brudne ręce, krwawa bluzka.

Odległość Levenshteina między tekstem początkowym a końcowym: 52
Znormalizowana odległość: 0.5149
Wynik różni się znacząco od oryginału.


In [38]:
## ZAD 5 
# 1. Wczytaj tekst z pliku PDF za pomocą PyPDF2
from PyPDF2 import PdfReader

pdf_file_path = 'PRZYKLADOWYPDFEN.pdf'
output_txt_path = 'PRZYKLADOWYPDFEN_pypdf2.txt'

extracted_text = ""
try:
    reader = PdfReader(pdf_file_path)
    print(f"Odczytano plik PDF: {pdf_file_path} (Liczba stron: {len(reader.pages)})")
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            print(f"  Odczytano tekst ze strony {i+1}...")
            extracted_text += text + "\n" # Dodaj nową linię między stronami
        else:
            print(f"  Strona {i+1} nie zawierała możliwego do wyekstrahowania tekstu (może być obrazem?).")

except FileNotFoundError:
    print(f"BŁĄD: Plik PDF '{pdf_file_path}' nie został znaleziony.")
    # Zakończ, jeśli plik nie istnieje
    raise
except Exception as e:
    print(f"BŁĄD podczas odczytu pliku PDF: {e}")
    # Zakończ w razie innych błędów odczytu
    raise


Odczytano plik PDF: PRZYKLADOWYPDFEN.pdf (Liczba stron: 1)
  Odczytano tekst ze strony 1...


In [37]:
from transformers import MarianMTModel
source_lang = "eng"
target_lang = "pol"

model_name = f"Allegro/BiDi-{source_lang}-{target_lang}"

translated_text = ""
if extracted_text:
    print("\nRozpoczynanie tłumaczenia...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        text = f">>{target_lang}<<" + " " + extracted_text

        batch_to_translate = [text]
        translations = model.generate(**tokenizer.batch_encode_plus(batch_to_translate, return_tensors="pt"))
        translated_text = tokenizer.batch_decode(translations, skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]

        print("Tłumaczenie zakończone.")

    except Exception as e:
        print(f"BŁĄD podczas tłumaczenia: {e}")

# 3. Zapisz wynik do pliku tekstowego
if translated_text:
    try:
        with open(output_txt_path, 'w', encoding='utf-8') as f:
            f.write(translated_text)
        print(f"\nPrzetłumaczony tekst został zapisany do pliku: {output_txt_path}")
        print("\n--- Przetłumaczony tekst (początek) ---")
        print(translated_text[:500] + "...")
        print("--- Koniec fragmentu ---")
    except Exception as e:
        print(f"BŁĄD podczas zapisu do pliku '{output_txt_path}': {e}")
elif not extracted_text:
     print("\nNie zapisano wyniku, ponieważ nie udało się wyekstrahować tekstu z PDF.")
else:
     print("\nNie zapisano wyniku, ponieważ wystąpił błąd podczas tłumaczenia.")


Rozpoczynanie tłumaczenia...
Tłumaczenie zakończone.

Przetłumaczony tekst został zapisany do pliku: PRZYKLADOWYPDFEN_pypdf2.txt

--- Przetłumaczony tekst (początek) ---
Jest to pierwsza strona przykładowego dokumentu PDF. Zawiera on jakiś tekst w języku angielskim. Rozpakujemy ten tekst za pomocą PyPDF2 i przetłumaczymy go. Dodajmy kolejne zdanie za dobrą miarę. Naturalne przetwarzanie języka jest fascynujące....
--- Koniec fragmentu ---
